# Introducción

La producción de frutas y cultivos en todo el mundo está muy influenciada por diversas enfermedades. Una disminución en la producción conduce a una degradación económica de la industria agrícola en todo el mundo. Los manzanos se cultivan en todo el mundo, y la manzana es una de las frutas más consumidas del mundo.

Exploraremos múltiples técnicas de pre-procesamiento de imágenes para mejorar la visibilidad de indicadores patológicos en plantas. 

# Exploración de datos

Primero, veamos la estructura del conjunto de datos y una descripción estadística de este, con el fin de conocer un poco más de las imágenes de las plantas que presentan o no enfermedades y así determinar cómo mejorar su visibilidad.

## Importación de las librerías necesarias

Importamos las librerías necesarias para leer los datos, explorarlos y procesarlos.

In [ ]:
import os

import pandas as pd
import cv2
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

from kaggle_datasets import KaggleDatasets

Definimos algunos parámetros necesarios para acceder a los datos y a para explorar las imágenes.

In [ ]:
TRAIN_PATH = "../input/plant-pathology-2020-fgvc7/train.csv"
TEST_PATH = "../input/plant-pathology-2020-fgvc7/test.csv"
IMAGE_PATH = "../input/plant-pathology-2020-fgvc7/images/"
IMAGE_SIZE = 512
SEED = 2022

Leemos los datos de entrenamiento para acceder a las imágenes mediante el identificador de la imagen *image_id*. Vemos también un poco la estructura de los datos, con las siguientes columnas:
- ***healthy***: Indica si la planta en la imagen está sana.
- ***multiple_diseases***: Indica si la planta presenta múltiples enfermedades.
- ***rust***: Indica si la planta presenta el síntoma de "oxidación" en sus hojas.
- ***scab***: Indica si la planta tiene el síntoma de "costra" en sus hojas.

In [ ]:
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
# Extraemos las etiquetas
train_labels = train_data.loc[:, 'healthy':].values

train_data.head()

## Carga de imágenes en memoria

Carguemos las imágenes de nuestro dataset para dar un vistazo general a las plantas presentes en éste y así plantear estrategias para procesar tales imágenes con las metodologías vistas en clase.

In [ ]:
def load_image(image_id):
    """Carga una imagen del dataset basado en su id.

    Args:
        image_id (str): Id. de la imagen en el dataset

    Returns:
        numpy.ndarray: Imagen cargada con cv2.cvtColor como ndarray
    
    """
    
    file_path = image_id + ".jpg" # Construímos la ruta
    image = cv2.imread(IMAGE_PATH + file_path) # Leemos la imagen
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA) # Reducimos el tamaño
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Retornamos la imagen en formato RGB

In [ ]:
healthy_data = train_data.query("healthy == 1") # Consultamos las plantas sanas
scab_data = train_data.query("scab == 1") # Consultamos las plantas con scab
rust_data = train_data.query("rust == 1") # Consultamos las plantas con rust
multiple_data = train_data.query("multiple_diseases == 1") # Consultamos las plantas con multiple_diseases

train_images = train_data["image_id"][:].apply(load_image) # Cargamos las imágenes de todas las plantas
healthy_images = train_images.loc[list(healthy_data.index)] # Cargamos las imágenes de las plantas sanas
scab_images = train_images.loc[list(scab_data.index)] # Cargamos las imágenes de las plantas con scab
rust_images = train_images.loc[list(rust_data.index)] # Cargamos las imágenes de las plantas con rust
multiple_images = train_images.loc[list(multiple_data.index)] # Cargamos las imágenes de las plantas con multiple_diseases

## Visualización de las imágenes de hojas

In [ ]:
def show_sample(sample):
    """
    Muestra un grid 4x4, con 4 imágenes de cada
    tipo de imagen en el dataset.
    
    Args:
        sample (list): Lista de imágenes.
        
    Returns:
        
    """
    # Visualización
    rows = ["Healthy", "Scab", "Rust", "Multiple diseases"]

    fig,ax = plt.subplots(nrows=4, ncols=4, figsize=(15, 15))
    for row in range(4):
        for col in range(4):
            ax[row,col].imshow(sample[row*4+col])
            ax[row, col].set_title("{}".format(rows[row]))
            ax[row,col].set_xticks([])
            ax[row,col].set_yticks([])

Una vez cargados los datos, veamos algunas de las imágenes que están presentes en el conjunto de datos.

In [ ]:
healthy_sample = healthy_images.iloc[0:4]
scab_sample = scab_images.iloc[0:4]
rust_sample = rust_images.iloc[0:4]
multiple_sample = multiple_images.iloc[0:4]

sample_images = [*healthy_sample, *scab_sample, *rust_sample, *multiple_sample]

In [ ]:
show_sample(sample_images)

## Distribución de canales

Una hoja sana es, usualmente, **verde**. Por otro lado, las manchas y costras que se presentan en una planta enferma suelen ser de **color marrón**, el cual en el modelo RGB es un color compuesto a partir del **verde y el rojo**, con menor apariencia del azul. Para ayudarnos un poco, usemos la librería Plotly para ver interactivamente los valores RGB en diferentes puntos de la imagen.

### Planta sana

Primero, veamos la imagen de la hoja de una planta sana.

In [ ]:
img = healthy_images.iloc[0]

fig = make_subplots(1, 2)

# Mostramos la imagen en la primera celda
fig.add_trace(go.Image(z=img), 1, 1)

# Mostramos el histograma de frecuencias (niveles)
# para cada canal RGB.
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)

fig.show()

En esta planta sana, el color **verde** es el predominante en intensidad.

### Planta con _rust_

El nombre _rust_ viene del color visto en las manchas de las hojas, el cual es un marrón _oxidado_ o amarillo. Podemos suponer que al ser de color marrón, veremos mucho más intensidad de rojos.

In [ ]:
img = rust_images.iloc[0]

fig = make_subplots(1, 2)

# Mostramos la imagen en la primera celda
fig.add_trace(go.Image(z=img), 1, 1)

# Mostramos el histograma de frecuencias (niveles)
# para cada canal RGB.
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)

fig.show()

Comparado con la planta sana, vemos mucho más frecuente el **rojo** con mayor intensidad (~120 en este caso). También es posible observar que los bordes amarillos reducen el valor del **azul**, mientras que las manchas marrones más desarrolladas reducen la intensidad del **verde**.

### Planta con _scab_

Estas enfermedades son producidas por hongos o bacterias y producen _costras_ o _scabs_. Estas costras suelen ser de color marrón o casi negras, pero no suelen ser amarillas.

In [ ]:
img = scab_images.iloc[0]

fig = make_subplots(1, 2)
# We use go.Image because subplots require traces, whereas px functions return a figure
fig.add_trace(go.Image(z=img), 1, 1)
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)
    
fig.show()

En el marrón encontrado en el _scab_, podemos ver que el **azul** y el **rojo** suelen subir en estas costras. Con esto, podemos ir generando hipótesis acerca de los patrones que se pueden encontrar en este tipo de enfermedades y por lo tanto es posible que, enfocándonos en el azul y el rojo de esta imagen, podamos determinar con más facilidad si la planta tiene _scab_.

### Plantas con múltiples enfermedades

In [ ]:
img = multiple_images.iloc[0]

fig = make_subplots(1, 2)
# We use go.Image because subplots require traces, whereas px functions return a figure
fig.add_trace(go.Image(z=img), 1, 1)
for channel, color in enumerate(['red', 'green', 'blue']):
    fig.add_trace(go.Histogram(x=img[..., channel].ravel(), opacity=0.5,
                               marker_color=color, name='%s channel' %color), 1, 2)
    
fig.show()

En este caso, las manchas pequeñas tienen mayor intensidad de rojos comparado con la parte sana de la hoja. El azul también sube en intensidad en la mayoría de estas manchas.

### Canales con valores medios

Vistos ya algunos ejemplos de las plantas que podemos encontrar en nuestro conjunto de datos, hagamos lo mismo para las medias del mismo.

In [ ]:
# Sacamos las medias de cada canal para luego realizar los histogramas

red_values = [np.mean(train_images[idx][:, :, 0]) for idx in range(len(train_images))]
green_values = [np.mean(train_images[idx][:, :, 1]) for idx in range(len(train_images))]
blue_values = [np.mean(train_images[idx][:, :, 2]) for idx in range(len(train_images))]
values = [np.mean(train_images[idx]) for idx in range(len(train_images))]

In [ ]:
fig = ff.create_distplot([values], group_labels=["Canales"], colors=["purple"])

fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Distribución de los valores de los canales")

fig.data[0].marker.line.width = 0.5

fig.show()

Podemos ver que la intensidad de los valores se distribuye aproximádamente normal con centro en ~105. Vemos algunos valores extremos en 60 y 145.

In [ ]:
fig = ff.create_distplot([red_values, green_values, blue_values], group_labels=["R", "G", "B"], colors=["red", "green", "blue"])

fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Distribución de los valores de rojo")

fig.data[0].marker.line.width = 0.5

fig.show()

Como se puede ver en los histogramas, los valores más frecuentes son los verdes, seguido de los rojos y los azules respectivamente. Tanto el canal azul como el canal rojo tienen una distribución aproximadamente normal, con centros en ~75 y ~100 respectivamente. Por otro lado, el canal verde parece una distribución asimétrica negativa.

En cualquier caso, los histogramas son similares entre sí, con una traslación para cada canal. 

# Pre-procesamiento

Para resaltar las características de las hojas de las plantas, es necesario realizar un debido preprocesamiento. Al resaltar las características del objeto de interés, se podrá mejorar la precisión de los modelos que se usen eventualmente.

Dentro de los métodos de pre-procesamiento usaremos filtros de suavizado, umbrales, transformaciones de color, etc.

## Detección de bordes de Canny

La detección de bordes con el algoritmo de Canny es un algoritmo que usa el gradiente de la intensidad de la imagen para determinar los bordes, a partir de los máximos locales. El umbral inferior y superior, sirven para descartar y seleccionar los verdaderos bordes de los que no se está tan seguro.

Veamos con mayor detalle cómo crearemos una máscara para nuestra imagen:

1. **Suavizado de la imagen:** El algoritmo de Canny es susceptible al ruido, por lo tanto es necesario suavizar la imagen para evitar bordes innecesarios.
2. **Conversión a espacio HSV:** Tras experimentar con el algoritmo de Canny en escala de grises y otros canales de múltiples espacios de color, obtuvimos los mejores resultados visibles con el espacio de color HSV, específicamente el canal de valor (V).
3. **Algoritmo de Canny:** Primero, se generan los valores de Sobel, los cuales describen el gradiente de la imagen. Con estos gradientes, detectamos los _posibles_ bordes. Para refinar los bordes, se aplica un doble umbral.


In [ ]:
def canny_and_mask(img, threshold=40):
    """
    Aplica el algoritmo de Canny para detección de bordes
    en una imagen en su canal V del espacio HSV.
    
    Args:
        img (np.ndarray): Imagen fuente a la que se le encontrarán los bordes.
        threshold (int): Umbral superior del algoritmo de Canny.
    """
    # Aplicamos un suavizado para reducir el ruido, pues
    # el algoritmo de Canny es susceptible al ruido.
    blur = cv2.GaussianBlur(img, (9, 9), 0)
    
    # Cambiar a espacio de color HSV y separar canales
    hsv = cv2.cvtColor(blur, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(hsv);
    
    # Aplicar Canny en canal V.
    edges = cv2.Canny(v, 0, threshold)
    
    # Dilatación para cerrar las líneas
    kernel = np.ones((5,5), np.uint8)
    edges_mask = cv2.dilate(edges, kernel, iterations=3)
    
    # Encontrar los contornos
    contours, _ = cv2.findContours(edges_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Encontramos el contorno más grande, que debería
    # corresponder a la hoja de la planta.
    biggest_cntr = None
    biggest_area = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > biggest_area:
            biggest_area = area
            biggest_cntr = contour
    
    # Una vez tenemos el contorno más grande,
    # lo dibujamos en nuestra máscara para luego
    # aplicarlo a la imagen original.
    mask = np.zeros_like(edges_mask);
    cv2.drawContours(mask, [biggest_cntr], -1, (255), -1);
    
    # Dilatamos y aplicamos un filtro de mediana para suavizar
    # y evitar bordes dentados.
    mask = cv2.erode(mask, kernel, iterations = 8)
    mask = cv2.dilate(mask, kernel, iterations = 8)
    mask = cv2.medianBlur(mask, 15)
    
    # Aplicamos la máscara a la imagen original
    masked = np.zeros_like(img);
    masked[mask == 255] = img[mask == 255]
    
    return masked

In [ ]:
masked_sample = [canny_and_mask(img) for img in sample_images]
show_sample(masked_sample)

Es de esperar que en las imágenes con muchas otras hojas cerca, los bordes no sean los deseados. Para combatir esto, tal vez se podría ajustar el valor de los umbrales uno por uno, o incluso usando el umbral de Otsu para el algoritmo de Canny como lo menciona [\[1\]][1], pero esto no resulto muy útil para este conjunto de datos, pues aunque algunas imágenes tenían fondos distinguidos, la mayoría de las imágenes no tenían un histograma bimodal, reduciendo la efectividad del umbral de Otsu.


---


- [\[1\] M. Fang, G. Yue, and Q. Yu, “The Study on An Application of Otsu Method in Canny Operator.”][1]


[1]: https://www.researchgate.net/publication/255634985_The_Study_on_An_Application_of_Otsu_Method_in_Canny_Operator

## Segmentación por color

### Con rango de color

Para las imágenes de las hojas, el algoritmo de Canny hizo un buen trabajo detectando los bordes de las imágenes y removiendo el fondo que no era de interés. Ahora, queremos remover las partes sanas de las hojas, esto es, remover la parte de verde de la hoja, dejando así solo las manchas o costras en la imagen.

In [ ]:
def mask_green(img, low_green=(36, 25, 25), high_green=(130, 255, 255)):
    """
    Enmascara los colores verde de una imagen.
    
    Args:
        img (np.ndarray): Imagen fuente que se va a enmascarar.
    
    Returns:
        masked (np.ndarray): Imagen enmascarada, con espacios negros
        donde se detectó el verde dentro del rango.
    """
    # Convertir a espacio de color hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv = cv2.GaussianBlur(hsv, (3, 3), 0)
    
    # Crear máscara basado en el rango
    green_mask = cv2.inRange(hsv, low_green, high_green)
    mask = cv2.bitwise_not(green_mask)
    
    # Cortar el verde
    imask = mask>0
    masked = np.zeros_like(img, np.uint8)
    masked[imask] = img[imask]
    return masked

In [ ]:
color_segmented_sample = [mask_green(img) for img in masked_sample]
show_sample(color_segmented_sample)

### Con único umbral

Intentamos encerrar en un rectángulo los posibles parches de enfermedad basado en el color.

In [ ]:
def unhealthy_selection(img, lower_thresh=20, upper_thresh=255):
    """
    Encierra lo que cree que puede ser una enfermedad basado en un umbral.
    Args:
        img (np.ndarray): Imagen fuente
        lower_thresh (int): Umbral inferior
        upper_thresh (int): Umbral superior
    Returns:
        copy (np.ndarray): Imagen con los contornos de los parches.
    """

    copy = img.copy()
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h, s, v = cv2.split(hsv)


    #plt.imshow(img_H, cmap='gray', aspect='auto');
    #plt.show()

    gray = cv2.threshold(h, lower_thresh, upper_thresh, cv2.THRESH_BINARY)[1]
    contours, hierarchy = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 10 < area < 1000:
            #cv2.drawContours(crop,[cnt],0,(255,0,0),2)
            x, y, w, h = cv2.boundingRect(cnt) # offsets - with this you get 'mask'
            #cv2.rectangle(crop,(x,y),(x+w,y+h),(255,0,0),0)
            average_color = np.array(cv2.mean(img[y:y+h,x:x+w])).astype(np.uint8)

            B_validation = 90<average_color[0]<255
            G_validation = 90<average_color[1]<255
            R_validation = 20<average_color[2]<90

            if(B_validation and G_validation and R_validation):
                #plt.imshow(crop[y:y+h,x:x+w])
                #plt.show()
                #print(average_color)
                cv2.rectangle(copy, (x,y), (x+w,y+h), (255,0,0), 2)
                #plt.hist(crop[y:y+h,x:x+w][:,:,2].ravel(), bins=256, range=(0.0, 255.0))
                #plt.show()

    return copy

In [ ]:
unlheathy_bounds = [unhealthy_selection(img) for img in masked_sample]
show_sample(unlheathy_bounds)

## Realce de contraste

Basado en las anotaciones que hicimos previamente, el color azul y el color rojo suben en intensidad en los puntos de enfermedad, mientras que el color verde disminuye. Intentemos realzar el contraste entre estas zonas. Para esto, usaremos la Ecualización de Histograma Adaptativo Limitado por Contraste (CLAHE por sus siglas en inglés).

La ecualización de histograma adaptativo genera histogramas de múltiples regiones de la imagen y distribuye de mejor manera los valores. Como esto puede generar ruido, se limita por contraste para evitar que se sobreamplifique. 

In [ ]:
def clahe(img):
    """
    Aplica Ecualización de Histograma Adaptativo Limitado por Contraste (CLAHE)
    a una imágen en sus canales HSV.
    
    Args:
        image (np.ndarray): Imagen de entrada
        
    Returns:
        clahe_image (np.ndarray): Imagen tras CLAHE
    """
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(3, 3))
    
    # Convertimos a HSV
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Separamos cada canal de HSV
    h, s, v = cv2.split(img)
    
    # Aplicamos el CLAHE.
    h = clahe.apply(h)
    s = clahe.apply(s)
    v = clahe.apply(v)
    
    hsv = cv2.merge((h, s, v))
    # BGR hace que las manchas se vean de color cyan
    # en la enfermedad Rust.
    clahe_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        
    return clahe_image

In [ ]:
clahe_sample = [clahe(img) for img in masked_sample]

In [ ]:
show_sample(clahe_sample)

# Conclusiones

- El conjunto de datos no es uniforme y por lo tanto es difícil hacer un preprocesamiento adecuado para todas las imágenes.
- La segmentación también se ve afectada por el ruido de las imágenes en sus fondos
- Las redes neuronales convolucionales como DenseNet o EfficientNetB7 son efectivas sin necesidad de segmentar tan arduamente.
- La extracción de características de textura se podrían hacer con matrices de Co ocurrencias, pero las redes CNN evitan la extracción manual de características y usualmente obtienen mejores resultados.
- El espacio de color HSV fue el más útil, pues permitía separar el Hue (matiz) y así enmascarar el color verde de la planta.
- El canal de valor de HSV también es útil para la umbralización.

# Referencias

[1]	P. Bansal, R. Kumar, and S. Kumar, “Disease detection in apple leaves using deep convolutional neural network,” Agriculture (Switzerland), vol. 11, no. 7, Jul. 2021, doi: 10.3390/agriculture11070617.

[2]	Adhiparasakthi Engineering College, Institute of Electrical and Electronics Engineers. Madras Section, and Institute of Electrical and Electronics Engineers, Proceedings of the 2019 IEEE International Conference on Communication and Signal Processing (ICCSP) : 4th - 6th April 2018, Melmaruvathur, India. 

[3]	Institute of Electrical and Electronics Engineers. Madras Section and Institute of Electrical and Electronics Engineers, 2019 5th International Conference on Advanced Computing & Communication Systems (ICACCS). 

[4]	V. Rajesh Kumar, K. Pradeepan, S. Praveen, M. Rohith, and V. Vasantha Kumar, “Identification of Plant Diseases Using Image Processing and Image Recognition,” Jul. 2021. doi: 10.1109/ICSCAN53069.2021.9526493.

[5]	P. Kulkarni, A. Karwande, T. Kolhe, S. Kamble, A. Joshi, and M. Wyawahare, “Plant Disease Detection Using Image Processing and Machine Learning.”

[6]	R. Anand, S. Veni, and J. Aravinth, “An application of image processing techniques for detection of diseases on brinjal leaves using k-means clustering method,” Sep. 2016. doi: 10.1109/ICRTIT.2016.7569531.

[7]	S. Zhang, Z. You, and X. Wu, “Plant disease leaf image segmentation based on superpixel clustering and EM algorithm,” Neural Computing and Applications, vol. 31, pp. 1225–1232, Feb. 2019, doi: 10.1007/s00521-017-3067-8.

[8]	IEEE Staff, 2018 International Conference on Electrical, Electronics, Communication, Computer, and Optimization Techniques (ICEECCOT). IEEE, 2018.

[9]	V. Singh and A. K. Misra, “Detection of plant leaf diseases using image segmentation and soft computing techniques,” Information Processing in Agriculture, vol. 4, no. 1, pp. 41–49, Mar. 2017, doi: 10.1016/j.inpa.2016.10.005.

[10]	S. S. Lomte and A. P. Janwale, “Plant Leaves Image Segmentation Techniques: A Review,” Article in INTERNATIONAL JOURNAL OF COMPUTER SCIENCES AND ENGINEERING, 2017, [Online]. Available: www.ijcseonline.org

[11] Amity University, Institute of Electrical and Electronics Engineers. United Kingdom and Republic of Ireland Section, and Institute of Electrical and Electronics Engineers, Abstract proceedings of International Conference on Automation, Computational and Technology Management (ICACTM-2019) : 24th - 25th April 2019. 
